# 1. 层和块


① nn.Sequential 定义了一种特殊的 Module。


In [ ]:
# 回顾一下多层感知机
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0589,  0.0950, -0.3810,  0.3201, -0.0735,  0.0783, -0.0542,  0.1098,
          0.3020,  0.2651],
        [ 0.0803, -0.0171, -0.4165,  0.1557, -0.0062, -0.1165, -0.1585,  0.0963,
          0.3906,  0.3161]], grad_fn=<AddmmBackward0>)

# 2. 自定义块


In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  # 调用父类的__init__函数
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


# 实例化多层感知机的层，然后在每次调用正向传播函数调用这些层
net = MLP()
X = torch.rand(2, 20)
print(X)
net(X)

tensor([[0.8892, 0.9472, 0.9177, 0.5405, 0.2315, 0.7996, 0.4348, 0.2758, 0.1721,
         0.1725, 0.0679, 0.6477, 0.1216, 0.9093, 0.7149, 0.6008, 0.2016, 0.8877,
         0.3484, 0.3399],
        [0.7259, 0.1185, 0.8537, 0.5033, 0.5242, 0.7212, 0.6489, 0.8308, 0.8414,
         0.2973, 0.4154, 0.1776, 0.5667, 0.0207, 0.3389, 0.5983, 0.7247, 0.3002,
         0.3414, 0.6420]])


tensor([[ 0.0200, -0.1409,  0.0435, -0.0940, -0.0834,  0.0601,  0.0595, -0.2327,
         -0.0117,  0.0016],
        [-0.0280,  0.0870, -0.0140, -0.0667, -0.0531, -0.0780, -0.1020, -0.0218,
          0.0245, -0.0491]], grad_fn=<AddmmBackward0>)

# 3. 顺序块


In [ ]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = (
                block  # block 本身作为它的key，存在_modules里面的为层，以字典的形式
            )

    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[-0.0651,  0.0377, -0.0348, -0.0377,  0.1602,  0.0022, -0.0904,  0.1742,
         -0.0520,  0.0189],
        [-0.0192,  0.1056, -0.0497,  0.0301,  0.2464,  0.0126, -0.1700,  0.4147,
          0.0703, -0.0013]], grad_fn=<AddmmBackward0>)

# 4. 正向传播


In [ ]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        # 代码做了什么：
        # 1. 随机初始化了一个20x20的矩阵
        # 2. 线性层的输入是20维的输入，输出是20维的输出
        # 3. 随机权重矩阵与线性层的输出相乘，得到一个20x20的矩阵
        # 4. ReLU激活函数
        # 5. 第二个线性层的输入是20维的输入，输出是20维的输出
        # 6. 第二个线性层的输出与随机权重矩阵相乘，得到一个20x20的矩阵
        # 7. 重复步骤4-6，直到矩阵的绝对值之和小于1
        # 8. 返回矩阵的元素之和
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight + 1))
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
X = torch.rand(2, 20)
net(X)

tensor(-0.0974, grad_fn=<SumBackward0>)

# 5. 混合组合块


In [ ]:
# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), 
            nn.ReLU(), 
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimear = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
X = torch.rand(2, 20)
chimear(X)

tensor(-0.1488, grad_fn=<SumBackward0>)

# 6. 参数管理


In [7]:
# 首先关注具有单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8), 
    nn.ReLU(), 
    nn.Linear(8, 1)
    )
X = torch.rand(size=(2, 4))
print(net(X))
print(net[2].state_dict())  # 访问参数，net[2]就是最后一个输出层
print(type(net[2].bias))  # 目标参数
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None)  # 还没进行反向计算，所以grad为None
print(
    *[(name, param.shape) for name, param in net[0].named_parameters()]
)  # 一次性访问所有参数
print(
    *[(name, param.shape) for name, param in net.named_parameters()]
)  # 0是第一层名字，1是ReLU，它没有参数
print(net.state_dict()["2.bias"].data)  # 通过名字获取参数

tensor([[-0.1499],
        [-0.1420]], grad_fn=<AddmmBackward0>)
OrderedDict([('weight', tensor([[ 0.3032, -0.0881, -0.0380,  0.3425,  0.0758, -0.1387,  0.1212, -0.1509]])), ('bias', tensor([-0.3423]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3423], requires_grad=True)
tensor([-0.3423])
True
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([-0.3423])


# 7. 嵌套块


In [ ]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(
            f"block{i}", block1()
        )  # f'block{i}' 可以传一个字符串名字过来，block2可以嵌套四个block1
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet(X))
print(rgnet)

tensor([[-0.1750],
        [-0.1750]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 8 内置初始化


In [ ]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))


def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 下划线表示把m.weight的值替换掉
        nn.init.zeros_(m.bias)


net.apply(init_normal)  # 会递归调用 直到所有层都初始化
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([ 0.0012, -0.0112, -0.0153,  0.0218])
tensor(0.)


In [ ]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))


def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)


net.apply(init_constant)
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([1., 1., 1., 1.])
tensor(0.)


In [ ]:
# 对某些块应用不同的初始化
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0479, -0.1771,  0.5267, -0.0020])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


# 9. 参数替换


In [ ]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init", *[(name, param.shape) for name, param in m.named_parameters()][0]
        )  # 打印名字是啥，形状是啥
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= (
            m.weight.data.abs() >= 5
        )  # 这里*=的代码相当于先计算一个布尔矩阵(先判断>=)，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素。保留绝对值大于5的权重，不是的话就设为0


net.apply(my_init)
print(net[0].weight[:2])
net[0].weight.data[:] += 1  # 参数替换
net[0].weight.data[0, 0] = 42
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[ 0.0000,  7.1240,  0.0000,  5.1135],
        [-8.6745, -7.3974,  0.0000, -0.0000]], grad_fn=<SliceBackward0>)
tensor([42.0000,  8.1240,  1.0000,  6.1135])


# 10. 参数绑定


In [ ]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1)
)  # 第2个隐藏层和第3个隐藏层是share权重的，第一个和第四个是自己的
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 11. 自定义层


In [ ]:
# 构造一个没有任何参数的自定义层
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()


layer = CenteredLayer()
print(layer(torch.FloatTensor([1, 2, 3, 4, 5])))

# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
print(Y.mean())


# 带参数的图层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(
            torch.randn(in_units, units)
        )  # nn.Parameter使得这些参数加上了梯度
        self.bias = nn.Parameter(
            torch.randn(
                units,
            )
        )

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


dense = MyLinear(5, 3)
print(dense.weight)

# 使用自定义层直接执行正向传播计算
print(dense(torch.rand(2, 5)))
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
print(net(torch.rand(2, 64)))

tensor([-2., -1.,  0.,  1.,  2.])
tensor(-6.2864e-09, grad_fn=<MeanBackward0>)
Parameter containing:
tensor([[-2.8449,  0.1887,  0.7945],
        [ 0.4226,  1.6180, -0.5880],
        [-0.4794, -0.0817, -0.3648],
        [-0.1979,  0.8702, -0.3515],
        [-1.4943,  0.3618,  0.2969]], requires_grad=True)
tensor([[0.0000, 0.0000, 1.3957],
        [0.8225, 0.0000, 0.9089]])
tensor([[0.],
        [0.]])


# 12. 读写文件


In [ ]:
# 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, "x-file")
x2 = torch.load("x-file")
print(x2)

# 存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x, y], "x-files")
x2, y2 = torch.load("x-files")
print(x2)
print(y2)

# 写入或读取从字符串映射到张量的字典
mydict = {"x": x, "y": y}
torch.save(mydict, "mydict")
mydict2 = torch.load("mydict")
print(mydict2)

tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3])
tensor([0., 0., 0., 0.])
{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}


In [ ]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))


net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(), "mlp.params")

# 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数
clone = MLP()  # 必须要先声明一下，才能导入参数
clone.load_state_dict(torch.load("mlp.params"))
print(clone.eval())  # eval()是进入测试模式

Y_clone = clone(X)
print(Y_clone == Y)

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
